<a href="https://colab.research.google.com/github/SaeidDaliriSusefi/sd/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
import geemap
import folium
from folium import plugins
from IPython.display import display, clear_output
import ipywidgets as widgets
import json
from shapely.geometry import shape
import os
from google.colab import files

In [2]:
ee.Authenticate()
ee.Initialize(project="ee-saeiddalirisu", opt_url='https://earthengine-highvolume.googleapis.com')

In [18]:
# Step 2: Add EE Layer to Folium

def add_ee_layer(self, ee_image_object, vis_params, name="Layer"):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer

jet_palette = [
    '#00007F', '#0000FF', '#007FFF', '#00FFFF',
    '#7FFF7F', '#FFFF00', '#FF7F00', '#FF0000', '#7F0000'
]

# Step 3: Define Widgets

dropdown = widgets.Dropdown(
    options=[],
    value=None,
    description='Return Period:',
    style={'description_width': 'initial'}
)
run_button = widgets.Button(description="Add Flood Layer", button_style='primary')
manual_upload_button = widgets.Button(description="📤 Upload ROI", button_style='info')
export_button = widgets.Button(description="📥 Export Flood Map (GeoTIFF)", button_style='success')

control_panel = widgets.VBox([
    widgets.HBox([dropdown, run_button, manual_upload_button]),
    export_button
])
display(control_panel)

# Step 4: Initialize Map

def initialize_map():
    m_init = folium.Map(location=[45.1, 11.2], zoom_start=8, control_scale=True)
    folium.TileLayer('OpenStreetMap').add_to(m_init)
    folium.TileLayer('CartoDB positron').add_to(m_init)
    folium.TileLayer(
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr='Esri', name='Esri Satellite', overlay=False, control=True
    ).add_to(m_init)
    folium.TileLayer(
        tiles='http://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr='Google', name='Google Satellite', overlay=False, control=True
    ).add_to(m_init)
    folium.TileLayer(
        tiles='https://gibs.earthdata.nasa.gov/wmts/epsg3857/best/BlueMarble_ShadedRelief_Bathymetry/default/2022-01-01/GoogleMapsCompatible_Level9/{z}/{y}/{x}.jpg',
        attr='NASA', name='NASA Blue Marble', overlay=False, control=True
    ).add_to(m_init)
    draw_control = plugins.Draw(export=True, position='topleft')
    m_init.add_child(draw_control)
    folium.LayerControl(position='topright').add_to(m_init)
    display(m_init)

initialize_map()

# Step 5: Global Variables
roi = None
geojson = None
center = [45.1, 11.2]
flood_image = None

# Step 6: Button Handlers

def on_button_click(b):
    global roi, geojson, center, flood_image
    if roi is None:
        print("⚠️ No ROI selected or uploaded.")
        return

    clear_output(wait=True)
    display(control_panel)

    rp = dropdown.value
    m2 = folium.Map(location=center, zoom_start=12, control_scale=True)

    for tile in [
        ('OpenStreetMap', None),
        ('CartoDB positron', None),
        ('Esri Satellite', 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'),
        ('Google Satellite', 'http://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}'),

    ]:
        name, url = tile
        folium.TileLayer(
            tiles=url if url else name,
            attr=name.split()[0],
            name=name,
            overlay=False,
            control=True
        ).add_to(m2)

    folium.GeoJson(
        data=geojson['features'][0]['geometry'],
        name='ROI Boundary',
        style_function=lambda x: {'color': 'black', 'fillOpacity': 0}
    ).add_to(m2)

    jrc = ee.ImageCollection("JRC/CEMS_GLOFAS/FloodHazard/v1")
    flood_image = jrc.filter(ee.Filter.eq('return_period', rp)).mosaic().clip(roi)
    stats = flood_image.reduceRegion(
        reducer=ee.Reducer.max(),
        geometry=roi,
        scale=30,
        maxPixels=1e9
    ).getInfo()
    vmax = stats.get('depth') or 1
    vis_params = {'min': 0.1, 'max': vmax, 'palette': jet_palette}
    m2.add_ee_layer(flood_image, vis_params, f"Flood Hazard RP {rp}")

    # ✅ Extract flooded area polygons
    try:
        flood_mask = flood_image.gt(0.1).selfMask()
        vectors = flood_mask.reduceToVectors(
            geometry=roi,
            scale=30,
            geometryType='polygon',
            labelProperty='flooded',
            reducer=ee.Reducer.countEvery()
        )
        flooded_geojson = vectors.getInfo()
        flooded_polygons = folium.GeoJson(
            data=flooded_geojson,
            name='Flooded Area (Polygon)',
            style_function=lambda x: {
                'color': 'blue',
                'fillColor': 'blue',
                'weight': 1,
                'fillOpacity': 0.3
            }
        )
        flooded_polygons.add_to(m2)
    except Exception as e:
        print("⚠️ Could not display flooded area polygons. Try a smaller ROI.")
        print(e)

    draw_control = plugins.Draw(export=True, position='topleft')
    m2.add_child(draw_control)

    legend_html = f'''
    <div style="position: fixed; bottom: 50px; left: 50px; width: 250px;
        background-color: white; border:2px solid black; z-index:9999;
        font-size:13px; padding:10px;">
        <b>Flood Hazard Legend<br>Return Period {rp}</b><br>
        <table style="border-collapse: collapse; width: 100%; margin-top: 10px;">
            <tr>
                <td style="background-color: #00007F; height: 12px;"></td>
                <td style="background-color: #0000FF;"></td>
                <td style="background-color: #007FFF;"></td>
                <td style="background-color: #00FFFF;"></td>
                <td style="background-color: #7FFF7F;"></td>
                <td style="background-color: #FFFF00;"></td>
                <td style="background-color: #FF7F00;"></td>
                <td style="background-color: #FF0000;"></td>
                <td style="background-color: #7F0000;"></td>
            </tr>
        </table>
        <div style="display: flex; justify-content: space-between; font-size: 12px;">
            <span>0.1 m</span><span>{vmax:.2f} m</span>
        </div>
    </div>
    '''
    m2.get_root().html.add_child(folium.Element(legend_html))
    folium.LayerControl(position='topright').add_to(m2)
    display(m2)

def on_export_click(b):
    try:
        rp = dropdown.value
        task = ee.batch.Export.image.toDrive(
            image=flood_image,
            description=f"FloodHazard_RP{rp}",
            folder='GEE_Flood_Exports',
            fileNamePrefix=f"Flood_RP{rp}",
            region=roi,
            scale=30,
            fileFormat='GeoTIFF',
            maxPixels=1e13
        )
        task.start()
        print(f"🚀 Export started: Flood_RP{rp}. Check your Google Drive → GEE_Flood_Exports folder.")
    except Exception as e:
        print("❌ Export failed. Make sure a flood map is generated before exporting.")
        print(e)

def on_manual_upload_click(b):
    print("📤 Select a ROI GeoJSON file to upload.")
    uploaded = files.upload()
    if uploaded:
        filename = list(uploaded.keys())[0]
        with open(filename, 'r') as f:
            geojson_data = json.load(f)
        global geojson, roi, center
        geojson = geojson_data
        roi = ee.Geometry(geojson['features'][0]['geometry'])
        center = shape(geojson['features'][0]['geometry']).centroid.coords[0][::-1]
        jrc = ee.ImageCollection("JRC/CEMS_GLOFAS/FloodHazard/v1").filterBounds(roi)
        return_periods = sorted(jrc.aggregate_array('return_period').distinct().getInfo())
        dropdown.options = return_periods
        dropdown.value = return_periods[0] if return_periods else None
        print("✅ ROI uploaded. You can now generate the flood map.")

run_button.on_click(on_button_click)
export_button.on_click(on_export_click)
manual_upload_button.on_click(on_manual_upload_click)

Test